In [3]:
import numpy as np
import h5py
import os
from scipy.ndimage import gaussian_filter1d

from nre import io, moco, roi
import ants

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

/Users/kyle/miniforge3/envs/imaging/lib/python3.12/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [4]:
def _zscore(x):
    """x: 2D array (n, t)"""

    x_mean = np.mean(x, axis=-1)
    x_std = np.std(x, axis=-1)
    return (x - x_mean[:, None]) / x_std[:, None]

def _zdff(F, win=200, smooth=False):
    """calculate zscored(df/f) based on F baseline activity"""

    # find average signal in first `win` volumes
    Fbase = np.mean(F[:, :win], axis=-1)
    dff = (F - Fbase[:, None]) / Fbase[:, None]

    if smooth:
        dff = gaussian_filter1d(dff, sigma=1)

    return _zscore(dff)

In [5]:
dropboxDir = "/Users/kyle/Library/CloudStorage/Dropbox-ahmedlab/ahmedlab-big/kyle/data"
fly = '20231211_101-002'
dataDir = os.path.join(dropboxDir, fly)
nifty_path = os.path.join(dataDir, fly+'_channel_1.nii')
labels_path = os.path.join(dataDir, fly+'_labels.h5')

In [6]:
brain = io.load(nifty_path)

n_slices = brain.shape[2]
n_vols = brain.shape[-1]

In [7]:
n_slices

59

In [8]:
n_vols

992

In [9]:
mean_brain = np.mean(brain[0:300], axis=-1)
fixed = ants.from_numpy(mean_brain)

In [21]:
io.save(os.path.join(dataDir, 'fixed.nii'), mean_brain)

In [22]:
moco_brain = np.zeros_like(brain)

for vol in range(n_vols):
    moving = ants.from_numpy(brain[:, :, :, vol])
    moco_brain[:, :, :, vol] = moco.apply(fixed, moving).numpy()

In [23]:
n_clusters = 500

In [27]:
io.save(os.path.join(dataDir, 'moco_brain.nii'), moco_brain)

In [6]:
moco_brain = io.load(os.path.join(dataDir, 'moco_brain.nii'))

In [7]:
moco_brain.shape

(253, 154, 59, 992)

In [56]:
%matplotlib qt
fig, ax = plt.subplots()
l = ax.imshow(restacked.T[:,:,0,0], origin = 'lower', cmap='binary')

axT = fig.add_axes([0.2, 0.95, 0.65, 0.03])
axO = fig.add_axes([0.2, 0.90, 0.65, 0.03])

sliderT = Slider(axT, 'z', 0, restacked.T.shape[2]-1, valinit=0, valfmt='%i')
sliderO = Slider(axO, 't', 0, restacked.T.shape[3]-1, valinit=0, valfmt='%i')

def update(val):
    i = int(sliderT.val)
    j = int(sliderO.val)
    im = restacked.T[:,:,i,j]
    l.set_data(im)
    fig.canvas.draw_idle()

sliderT.on_changed(update)
sliderO.on_changed(update)

plt.show()

In [28]:
xmfile = '/Users/kyle/Library/CloudStorage/Dropbox-ahmedlab/ahmedlab-big/kyle/data/20231211_101-002/20231211_101-002.xml'

In [29]:
import xml.etree.cElementTree as CT

In [32]:
tree = CT.parse(xmfile)

In [33]:
tree.findall("Frame/PVStateShard/PVStateValue/Bruker 400 μm Piezo")

[]

In [34]:
tree

In [35]:
338-164

174

In [36]:
174/3

58.0

In [38]:
(306-144)

162

In [39]:
nifty = io.load('/Users/kyle/Library/CloudStorage/Dropbox-ahmedlab/ahmedlab-big/kyle/data/20231211_101-002/20231211_101-002_channel_1.nii')

In [42]:
nifty.T.shape

(992, 59, 154, 253)

In [44]:
restacked = np.empty_like(nifty.T)

In [53]:
restacked = nifty.T[0:10].flatten()[0:22597960].reshape((-1,57,154,253))

In [50]:
restacks = np.reshape(nifty.T[0:10], (-1, 58,154,253), )

ValueError: cannot reshape array of size 22987580 into shape (58,154,253)

In [51]:
58*10*154*253

22597960

In [43]:
for t in nifty.T:
    print(t.shape)
    break

(59, 154, 253)


In [55]:
io.save('restacked.nii', restacked.T)

In [60]:
(35.572397694 - 35.562137412) * 1000

10.260282000004395

In [61]:
(35.675000514 - 35.572397694) * 1000

102.60281999999421